In [1]:
from git import Repo
import os
import re
import string
import gensim
import ast
from collections import Counter

In [3]:
# specify the list of repository urls to crawl
repository_list = ['https://github.com/matplotlib/matplotlib.git',
                    'https://github.com/scikit-learn/scikit-learn.git',
                    'https://github.com/numpy/numpy.git',
                    'https://github.com/scipy/scipy.git',
                    'https://github.com/pallets/flask.git',
                    'https://github.com/psf/requests.git',
                    'https://github.com/scrapy/scrapy']

# specify the path of the output text file
output_file = "python_files.txt"

# loop through all repository urls
for url in repository_list:
    # clone the repository to a temporary directory
    repo_dir = os.path.join(os.getcwd(), "temp")
    Repo.clone_from(url, repo_dir)

    # loop through all Python files in the repository
    for root, dirs, files in os.walk(repo_dir):
        for file in files:
            if file.endswith(".py"):
                # get the contents of the Python file
                file_path = os.path.join(root, file)
                with open(file_path, "r", encoding='utf-8') as f:
                    file_content = f.read()
                # write the contents of the Python file to the output file
                with open(output_file, "a", encoding='utf-8') as f:
                    f.write(f"Repository: {url}\nFile: {file_path}\n\n{file_content}\n\n{'-'*50}\n\n")

    # delete the temporary directory
    os.system(f"rm -rf {repo_dir}")

In [15]:
# Count the number of lines of code in this file
num_lines = 0

with open('python_files.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()
    num_lines = len(lines)

print(f"Lines: {num_lines}")

Lines: 1439996


In [28]:
# Creates a function that counts the number of words in output file
def tokenize(path):

    token_list = []

    # count the number of tokens in the file
    with open(path, "r", encoding='utf-8') as f:
        file_content = f.read()

    tokens = re.findall(r'\b\w+\b', file_content)
    token_list.append(tokens)
    num_tokens = len(tokens)
    print(f"Number of tokens: {num_tokens}")

    return token_list

tokenized_list = tokenize("python_files.txt")


Number of tokens: 5978486


In [42]:
# Train your Gensim Word2Vec model with the tokenized lines of code
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

model.build_vocab(tokenized_list)

model.train(tokenized_list, total_examples=model.corpus_count, epochs=model.epochs)


(50000, 29892430)

In [34]:
#Explored the trained model by examining the closest_words to “for”
model.wv.most_similar('for')

[('to', 0.9962059855461121),
 ('not', 0.9961488842964172),
 ('f', 0.996028482913971),
 ('in', 0.9957643151283264),
 ('the', 0.9957113862037659),
 ('name', 0.9956578612327576),
 ('return', 0.9954971075057983),
 ('if', 0.9953508377075195),
 ('sphinx', 0.9949526190757751),
 ('this', 0.9948042035102844)]

In [35]:
#Explored the trained model by examining the closest_words to “if”
model.wv.most_similar('if')

[('in', 0.9966590404510498),
 ('to', 0.9966531991958618),
 ('f', 0.9965929388999939),
 ('return', 0.9965319633483887),
 ('app', 0.9964976906776428),
 ('not', 0.9964142441749573),
 ('name', 0.9961705207824707),
 ('the', 0.9958152174949646),
 ('path', 0.9957109689712524),
 ('as', 0.9956626892089844)]

In [43]:
#Examining the popular identifier names like "math" and "numpy" and the similarity between them
model.wv.similarity('math','numpy')

0.32239005